In [8]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from more.datasets import ModulationDataSets
from more.network import MoreNet,init_weights
from more.train import train_center
from utils.centerloss import CenterLoss
import datetime
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [9]:
train_path = r'E:\train.h5'
test_path = r'E:\test.h5'
train_dataset = ModulationDataSets(train_path)
test_dataset = ModulationDataSets(test_path)
label_name = train_dataset.get_labels_name();

In [10]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)

In [4]:
net = MoreNet()
X = torch.rand(size=(1, 1, 196, 64), dtype=torch.float32)
net(X,True)

E:\workspace\DeepMore\more\network.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return x.detach(), F.softmax(y)


(tensor([[-0.0052,  0.0071]]),
 tensor([[0.1780, 0.0874, 0.1620, 0.1584, 0.0464, 0.1172, 0.1646, 0.0861]],
        grad_fn=<SoftmaxBackward0>))

In [5]:
net.load_state_dict(torch.load('basic-8.parm'))

<All keys matched successfully>

In [6]:
torch.save(net.state_dict(),"basic-8.parm")

In [11]:
init_weights(net)

In [12]:
loss= nn.CrossEntropyLoss()
center_loss=CenterLoss(num_classes=train_dataset.get_numclasses(),feat_dim=2,use_gpu=True)
lr=0.01
lr_center=0.5
opt_model=torch.optim.SGD(net.parameters(),lr,weight_decay=5e-04, momentum=0.9)
opt_center=torch.optim.SGD(center_loss.parameters(),lr_center)
net=net.to("cuda:0")

In [ ]:
start_time = time.time()

for epoch in range(50):
    print("==> Epoch {}/{}".format(epoch+1,50))

    train_center(net, loss, center_loss, opt_model, opt_center, train_loader, True, train_dataset.get_numclasses(), epoch)
    # if  (epoch+1) % 10 == 0 or (epoch+1) == 50:
    #     print("==> Test")
    #     acc, err = test(model, testloader, use_gpu, dataset.num_classes, epoch)
    #     print("Accuracy (%): {}\t Error rate (%): {}".format(acc, err))

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))

==> Epoch 1/50
Batch 50/313	 Loss 2.085363 (2.245551) XentLoss 2.084807 (2.080135) CenterLoss 0.000556 (0.165417)
Batch 100/313	 Loss 2.080625 (2.163029) XentLoss 2.079983 (2.079917) CenterLoss 0.000642 (0.083112)
Batch 150/313	 Loss 2.086701 (2.136124) XentLoss 2.085485 (2.080026) CenterLoss 0.001216 (0.056099)
Batch 200/313	 Loss 2.084760 (2.122507) XentLoss 2.081285 (2.079875) CenterLoss 0.003476 (0.042632)
Batch 250/313	 Loss 2.090072 (2.115164) XentLoss 2.080829 (2.079774) CenterLoss 0.009243 (0.035389)
Batch 300/313	 Loss 2.539413 (2.125277) XentLoss 2.071235 (2.079369) CenterLoss 0.468179 (0.045908)
==> Epoch 2/50
Batch 50/313	 Loss 3.949973 (3.079495) XentLoss 2.006993 (2.059629) CenterLoss 1.942980 (1.019866)
Batch 100/313	 Loss 9.316999 (4.912844) XentLoss 2.000646 (2.041368) CenterLoss 7.316353 (2.871476)
Batch 150/313	 Loss 54.967037 (12.200146) XentLoss 1.919086 (2.017095) CenterLoss 53.047951 (10.183050)


In [10]:
torch.save(net.state_dict(),"basic-8.parm")